In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/Colab Notebooks/craud/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

Пары подряд идущих запросов пользователей из таблицы пользовательских сессий:

In [ ]:
data = pd.read_csv(path + 'query.csv')
data

,id,query1,query2
0,y1000000361630744646-1631522512,Галицкий продал магнит,Автобус саранск тольятти
1,y1000000361630744646-1631522512,Метро москвы схема,tit
2,y1000000361630744646-1631522512,санино киевская расписание электричек,Метро москвы схема
3,y1000000361630744646-1631522512,голден палас,санино киевская расписание электричек
4,y1000000361630744646-1631522512,бизнес на уличных туалетах,голден палас
...,...,...,...
1475271,y9999969291608062519-1632251234,картины модульные,декоративная подушка
1475272,y9999969291608062519-1632251234,русские горки о чем сериал,картины модульные
1475273,y9999969291608062519-1632251234,сима ленд,русские горки о чем сериал
1475274,y9999969291608062519-1632251234,как правильно развесить настенный декор в инте...,сима ленд


In [ ]:
!pip install transformers

In [ ]:
!pip install pytorch_lightning

In [ ]:
import argparse
from typing import Dict
import os
import json
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler

from transformers import AutoModel, AutoTokenizer, AdamW

from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import EarlyStopping

os.environ["TOKENIZERS_PARALLELISM"] = "0"

class Embedder(nn.Module):
    def __init__(self, model_path, freeze_bert, layer_num):
        super().__init__()

        self.model = AutoModel.from_pretrained(model_path)
        self.model.trainable = not freeze_bert
        self.bert_dim = self.model.config.hidden_size
        self.layer_num = layer_num

    def forward(self, input_ids, attention_mask):
        output = self.model(
           input_ids,
           attention_mask=attention_mask,
           return_dict=True,
           output_hidden_states=True
        )
        layer_embeddings = output.hidden_states[self.layer_num]
        embeddings = torch.mean(layer_embeddings, dim=1)
        norm = embeddings.norm(p=2, dim=1, keepdim=True)
        embeddings = embeddings.div(norm)
        return embeddings


class ClusteringContrastiveModel(LightningModule):
    def __init__(self, model_path, freeze_bert=False, layer_num=-1, margin=0.5, lr=1e-5):
        super().__init__()

        self.embedder = Embedder(
            model_path,
            freeze_bert=freeze_bert,
            layer_num=layer_num
        )
        self.lr = lr
        self.loss = nn.CosineEmbeddingLoss(margin=margin)

    def forward(self, left, right, labels):
        left_embeddings = self.embedder(left["input_ids"], left["attention_mask"])
        right_embeddings = self.embedder(right["input_ids"], right["attention_mask"])
        loss = self.loss(left_embeddings, right_embeddings, labels)
        return loss

    def training_step(self, batch, batch_nb):
        train_loss = self(*batch)
        return train_loss

    def validation_step(self, batch, batch_nb):
        val_loss = self(*batch)
        self.log("val_loss", val_loss, prog_bar=True, logger=True)
        return val_loss

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.lr)
        return [optimizer]



class NewsDataset(Dataset):
    def __init__(self, records, model_path, max_tokens):
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_path
        )
        self.max_tokens = max_tokens
        self.records = records

    def __len__(self):
        return len(self.records)

    def __getitem__(self, index):
        fields = self.records[index]
        label = fields[-1]
        samples = fields[:-1]
        samples = [self.tokenizer(
            s,
            add_special_tokens=True,
            max_length=self.max_tokens,
            padding="max_length",
            truncation=True,
            return_tensors='pt'
        ) for s in samples]
        samples = [{key: value.squeeze(0) for key, value in s.items()} for s in samples]
        return samples[0], samples[1], torch.tensor(float(1 if label == 1 else -1))


def form_pairs(filename):
    records = []
    with open(filename, "r") as f:
        for line in f:
            record = json.loads(line)
            records.append((record["s1"], record["s2"], record["target"]))
    return records


def train_paraphrases(
    initial_model_name,
    train_path,
    max_tokens,
    out_dir,
    batch_size,
    grad_accum_steps,
    epochs,
    lr
):
    records = form_pairs(train_path)
    random.shuffle(records)
    border = int(len(records) * 0.8)
    train_records = records[:border]
    val_records = records[border:]

    train_data = NewsDataset(train_records, initial_model_name, max_tokens)
    train_sampler = RandomSampler(train_data)
    train_loader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    val_data = NewsDataset(val_records, initial_model_name, max_tokens)
    val_loader = DataLoader(val_data, batch_size=batch_size)

    LOG_EVERY_N_STEPS = 5
    model = ClusteringContrastiveModel(initial_model_name, lr=lr)
    early_stop_callback = EarlyStopping(
        monitor="val_loss",
        min_delta=0.0001,
        patience=3,
        verbose=True,
        mode="min"
    )
    trainer = Trainer(
        gpus=1,
        checkpoint_callback=True,
        accumulate_grad_batches=grad_accum_steps,
        max_epochs=epochs,
        callbacks=[early_stop_callback],
        log_every_n_steps=LOG_EVERY_N_STEPS
    )
    trainer.fit(model, train_loader, val_loader)
    train_data.tokenizer.save_pretrained(out_dir)
    model.embedder.model.save_pretrained(out_dir)

Загрузим данные, размеченные в Толоке:

In [ ]:
pool1 = pd.read_csv(path + 'result1.tsv', sep='\t')
pool2 = pd.read_csv(path + 'result2.tsv', sep='\t')

In [ ]:
df = pd.concat([pool1, pool2])

In [ ]:
df

,INPUT:session_id,INPUT:text1,INPUT:text2,OUTPUT:res,CONFIDENCE:res,Unnamed: 5
0,y5491971611630764813-1631723433,"беседы по социальному окружению ""кто нас воспи...",беседы по социальному окружению в подготовител...,1,100.00%,NaN
1,y4536280011629492713-1630602486,коснулся разбор слова по составу,чат рулетка по всему миру,0,100.00%,NaN
2,y3158831181592565381-1632230634,русь санаторий анапа официальный сайт,русь санаторий анапа отзывы,1,100.00%,NaN
3,y6962060371631071364-1631191948,государство 9 класс обществознание конспект урока,навигатор 55,0,100.00%,NaN
4,y4645798721619525366-1631695151,мид и вклад в социологю,джордж мид,0,100.00%,NaN
...,...,...,...,...,...,...
9985,y5378697651579646182-1631634228,электронной дневник московской области школа,математика 2класс 1часть учебник ответы стр 6,0,100.00%,NaN
9986,y7057059081618518771-1632331397,переливает бензин рено сафран 2.0,не заводится на холодную рено сафран,1,67.30%,NaN
9987,y5593890401604246131-1631555898,как правильно сделать договор аренды трактора ...,как рассчитать аренду трактора мтз 3522,1,98.48%,NaN
9988,y5004841251605798003-1632144248,нужна финансовая помощь,помогу деньгами сегодня,1,50.03%,NaN


In [ ]:
train = df.copy()
train.drop(columns=["INPUT:session_id", "CONFIDENCE:res", "Unnamed: 5"], inplace=True)
train.rename(columns = {'INPUT:text1':'s1', 'INPUT:text2':'s2', 'OUTPUT:res':'target'}, inplace = True)
train

,s1,s2,target
0,"беседы по социальному окружению ""кто нас воспи...",беседы по социальному окружению в подготовител...,1
1,коснулся разбор слова по составу,чат рулетка по всему миру,0
2,русь санаторий анапа официальный сайт,русь санаторий анапа отзывы,1
3,государство 9 класс обществознание конспект урока,навигатор 55,0
4,мид и вклад в социологю,джордж мид,0
...,...,...,...
9985,электронной дневник московской области школа,математика 2класс 1часть учебник ответы стр 6,0
9986,переливает бензин рено сафран 2.0,не заводится на холодную рено сафран,1
9987,как правильно сделать договор аренды трактора ...,как рассчитать аренду трактора мтз 3522,1
9988,нужна финансовая помощь,помогу деньгами сегодня,1


In [ ]:
train.to_csv(path + 'train.csv')

In [ ]:
# сохраним в train файл с jsonами
import json
train.to_json(path+'train.json', orient='records', lines=True)

In [ ]:
records = []
with open(path+'train.json', "r") as f:
  for line in f:
    record = json.loads(line)
    records.append((record["s1"], record["s2"], record["target"]))

# Обучение на всей разметке из Толоки и предсказание схожести для части неразмеченных пар запросов

Вначале обучим модель на всем размеченном датасете:

In [ ]:
# if __name__ == "__main__":
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--initial-model-name", type=str, default="DeepPavlov/rubert-base-cased")
#     parser.add_argument("--train-path", type=str, required=True)
#     parser.add_argument("--max-tokens", type=int, default=64)
#     parser.add_argument("--out-dir", type=str, required=True)
#     parser.add_argument("--batch-size", type=int, default=32)
#     parser.add_argument("--grad-accum-steps", type=int, default=8)
#     parser.add_argument("--epochs", type=int, default=5)
#     parser.add_argument("--lr", type=float, default=1e-05)
#     args = parser.parse_args()
#     train_paraphrases(**vars(args))


train_paraphrases(initial_model_name="DeepPavlov/rubert-base-cased",
                  train_path=path+'train.json',
                  max_tokens=64,
                  out_dir=path+'train_all/',
                  batch_size=32,
                  grad_accum_steps=8,
                  epochs=5,
                  lr=1e-05)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                | Params
-------------------------------------------------
0 | embedder | Embedder            | 177 M 
1 | loss     | CosineEmbeddingLoss | 0     
-------------------------------------------------
177 M     Trainable params
0         Non-trainable params
177 M     Total params
711.414   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Metric val_loss improved. New best score: 0.095


...

Metric val_loss improved by 0.002 >= min_delta = 0.0001. New best score: 0.086


Validating: 0it [00:00, ?it/s]

Metric val_loss improved by 0.001 >= min_delta = 0.0001. New best score: 0.085


Validating: 0it [00:00, ?it/s]

Metric val_loss improved by 0.000 >= min_delta = 0.0001. New best score: 0.085


Лучший скор на валидационной выборке получился: **0.085**

In [ ]:
model

ClusteringContrastiveModel(
  (embedder): Embedder(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)


Теперь с ее помощью можно предсказать схожесть подряд идущих запросов из таблицы сессий:

In [ ]:
test_all = data.copy()
test_all.drop(columns=["id"], inplace=True)
test_all.rename(columns = {'query1':'s1', 'query2':'s2'}, inplace = True)
test_all.to_json(path+'test_all.json', orient='records', lines=True)

Загрузим обученную модель:

In [ ]:
model_path = path + 'train_all/'
model = ClusteringContrastiveModel(model_path)

In [ ]:
records_test = []
with open(path+'test_all.json', "r") as f:
  for line in f:
    record = json.loads(line)
    record["target"] = 0
    if record["s1"] != None and record["s2"] != None:
      records_test.append((record["s1"], record["s2"], record["target"] ))

In [ ]:
len(records_test)

1474377

Поскольку датасет очень большой и содержит порядка **~1.4** миллиона пар, обработаем хотя бы **30000** из них:

In [ ]:
records_small = records_test[:30000]

In [ ]:
news_dataset = NewsDataset(records_small, model_path, 64)
test_loader = DataLoader(news_dataset, batch_size=1)

In [ ]:
res = []
prediction_list = []
for i, (l, r, t) in enumerate(test_loader):
  output = model(l, r, t)
  res.append((i, records_test[i][0], records_test[i][1], output.item()))
  prediction_list.append(output.item())

In [ ]:
res_df = pd.DataFrame(res, columns =['index', 's1', 's2', 'prob'])
res_df

,index,s1,s2,prob
0,0,Галицкий продал магнит,Автобус саранск тольятти,0.000000
1,1,Метро москвы схема,tit,0.000000
2,2,санино киевская расписание электричек,Метро москвы схема,0.084691
3,3,голден палас,санино киевская расписание электричек,0.000000
4,4,бизнес на уличных туалетах,голден палас,0.000000
...,...,...,...,...
29995,29995,держатель балконной двери,держатель балконной двери с широким,0.494550
29996,29996,может ли гипотеза быть ошибочной? как определи...,может ли гипотеза быть ошибочной? 7 класс физика,0.468876
29997,29997,назовите основные этапы метода научного познан...,может ли гипотеза быть ошибочной? как определи...,0.000000
29998,29998,может ли гипотеза быть ошибочной? как определи...,как определить истинность гипотезы? 7 класс фи...,0.381625


In [ ]:
res_df.to_csv(path+'res_probs_30000.csv')

,Unnamed: 0,index,s1,s2,prob
0,0,25001,что это за здание алабяна 15 с 2,продажа квартиры по Алабяна д 10 корп 3,0.179163
1,1,25002,что это за здание алабяна 10 с 3,что это за здание алабяна 15 с 2,0.498699
2,2,25003,что это за здание алабяна 15 с 2,что это за здание алабяна 10 с 3,0.498699
3,3,25004,если женщина сама себя нахваливает,дайте оскар этой богине песня кто поет,0.000000
4,4,25005,как наложить музыку на видео,лана роудс,0.000000


# Обучение и тестирование на разметке из Толоки

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(train, test_size=0.15, random_state=42)
train_df.to_json(path+'train_df.json', orient='records', lines=True)
test_df.to_json(path+'test_df.json', orient='records', lines=True)

In [ ]:
train_df[:2000].to_csv(path + 'train_small.csv')

In [ ]:
records = []
with open(path+'train_df.json', "r") as f:
  for line in f:
    record = json.loads(line)
    records.append((record["s1"], record["s2"], record["target"]))

In [ ]:
# if __name__ == "__main__":
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--initial-model-name", type=str, default="DeepPavlov/rubert-base-cased")
#     parser.add_argument("--train-path", type=str, required=True)
#     parser.add_argument("--max-tokens", type=int, default=64)
#     parser.add_argument("--out-dir", type=str, required=True)
#     parser.add_argument("--batch-size", type=int, default=32)
#     parser.add_argument("--grad-accum-steps", type=int, default=8)
#     parser.add_argument("--epochs", type=int, default=5)
#     parser.add_argument("--lr", type=float, default=1e-05)
#     args = parser.parse_args()
#     train_paraphrases(**vars(args))


train_paraphrases(initial_model_name="DeepPavlov/rubert-base-cased",
                  train_path=path+'train_df.json',
                  max_tokens=64,
                  out_dir=path+'train/',
                  batch_size=32,
                  grad_accum_steps=8,
                  epochs=5,
                  lr=1e-05)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                | Params
-------------------------------------------------
0 | embedder | Embedder            | 177 M 
1 | loss     | CosineEmbeddingLoss | 0     
-------------------------------------------------
177 M     Trainable params
0         Non-trainable params
177 M     Total params
711.414   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Metric val_loss improved. New best score: 0.099


...

Metric val_loss improved by 0.002 >= min_delta = 0.0001. New best score: 0.089


Validating: 0it [00:00, ?it/s]

Metric val_loss improved by 0.001 >= min_delta = 0.0001. New best score: 0.087


Validating: 0it [00:00, ?it/s]

Metric val_loss improved by 0.000 >= min_delta = 0.0001. New best score: 0.087


In [ ]:
model_test = ClusteringContrastiveModel(path + 'train/')

records = []
with open(path+'test_df.json', "r") as f:
  for line in f:
    record = json.loads(line)
    if record["s1"] != None and record["s2"] != None:
      records.append((record["s1"], record["s2"], record["target"] ))

In [ ]:
len(records)

2999

In [ ]:
news_dataset = NewsDataset(records, path + 'train/', 64)
test_loader = DataLoader(news_dataset, batch_size=1)

preds = []
y_true = []
for i, (l, r, t) in enumerate(test_loader):
  output = model_test(l, r, t)
  preds.append(output.item())
  y_true.append(records[i][2])

In [ ]:
preds_labeled = []
for i in preds:
  if i >= 0.5:
    preds_labeled.append(1)
  else:
    preds_labeled.append(0)

In [ ]:
from sklearn.metrics import mean_absolute_error, f1_score, accuracy_score

print('MAE score: ', mean_absolute_error(y_true, preds_labeled))

MAE score:  0.4551517172390797


In [ ]:
test_dataset = pd.read_csv(path+'test_dataset.csv')

In [ ]:
test_dataset

,Unnamed: 0,query1,query2
0,0,как царапать глаза,как сделать двойную веку
1,1,основные методы реабилитации пороках сердца,что надо сделать чтобы поднялась температура
2,2,кетонал инструкция по применению,тошнота как избавиться
3,3,лобода песня суперзвезда клип,как скачать песню на айфон
4,4,цинктерал стоимость в аптеке,если мало есть и заниматься спортом можно ли п...
...,...,...,...
495,495,почему армянки красивые,отучить котенка писать на кровать
496,496,борщевик сосновского история создания,как хранить деньги в кошельке
497,497,солнечный берег чувашия новый год,жирная кожа лица что делать
498,498,вязаная шапка из толстых ниток,как ломается голос у девочек


In [ ]:
records = []
for _, row in test_dataset.iterrows():
    records.append((row["query1"], row["query2"], 0))

In [ ]:
news_dataset = NewsDataset(records, path + 'train/', 64)
test_loader = DataLoader(news_dataset, batch_size=1)

preds = []
for i, (l, r, t) in enumerate(test_loader):
  output = model_test(l, r, t)
  preds.append(output.item())

In [ ]:
res_not_zero = []
res_all = []
for i in range(len(records)):
  print((i, records[i][0], records[i][1], preds[i]))
  res_all.append((i, records[i][0], records[i][1], preds[i]))
  if preds[i] > 0.03:
    res_not_zero.append((i, records[i][0], records[i][1], preds[i]))

(0, 'как царапать глаза', 'как сделать двойную веку', 0.02437150478363037)
(1, 'основные методы реабилитации пороках сердца', 'что надо сделать чтобы поднялась температура', 0.0)
(2, 'кетонал инструкция по применению', 'тошнота как избавиться', 0.0)
(3, 'лобода песня суперзвезда клип', 'как скачать песню на айфон', 0.0)
(4, 'цинктерал стоимость в аптеке', 'если мало есть и заниматься спортом можно ли похудеть', 0.0)
(5, 'создать аккаунт гугл', 'создать аккаунт gmail', 0.3481371998786926)
(6, 'перепелиные яйца польза и вред как принимать', 'как сварить рассыпчатый рис', 0.03172045946121216)
(7, 'virtualbox debian', 'как создать новый файл в терминале', 0.0)
(8, 'как правильно приготовить фрикадельки для супа', 'как правильно приготовить суп', 0.40471845865249634)
(9, 'информация по imei', 'как сбросить настройки айфона', 0.0)
(10, 'математика', 'учим английский язык с нуля', 0.08509963750839233)
(11, 'развитие мотивации профессиональной деятельности', 'мотивация', 0.189630389213562)
(12

In [ ]:
len(res_not_zero)

171

In [ ]:
for i in range(147, len(res_all)):
  if res_all[i][3] <= 0.1:
    ans.append((res_all[i][0], res_all[i][1], res_all[i][2], 0))
  elif res_all[i][3] >= 0.25:
    ans.append((res_all[i][0], res_all[i][1], res_all[i][2], 1))
  else:
    print(res_all[i][1], res_all[i][2], res_all[i][3])
    t = int(input())
    ans.append((res_all[i][0], res_all[i][1], res_all[i][2], t))

не убирается подчеркивание ссылок как в css убрать подчеркивание ссылки 0.12149858474731445
0
почта россии сроки доставки посылок по россии скорость доставки ems 0.1315937042236328
0
почта россии сроки доставки посылок по россии сколько стоит отправить посылку по россии 0.22536110877990723
0
праздники в июне 2018 как отдыхаем на новый год 2019 и выходные дни на январские праздники 0.22900229692459106
0
роблокс завести аккаунт в роблокс 0.24005568027496338
1
съедобные птицы что такое боровая дичь 0.1392974853515625
0
съедобные птицы когда охотиться на лесных птиц 0.11181670427322388
0
съедобные птицы можно ли есть голубей 0.1007007360458374
0
съедобные птицы краткий определитель птиц 0.12667042016983032
0
тульская чайная меню тула где в туле попить чай из самовара 0.2311849594116211
1
уголок фантазеры оформление школьного уголка в детском саду 0.16048967838287354
0
уголок фантазеры купить оформление для классного уголка 0.14982616901397705
0
уголок фантазеры как оформить классный уголок

In [ ]:
ans_df = pd.DataFrame(answers, columns =['', 'query1', 'query2', 'label'])
ans_df.drop(columns=[""], inplace=True)
ans_df

,query1,query2,label
0,как царапать глаза,как сделать двойную веку,0
1,основные методы реабилитации пороках сердца,что надо сделать чтобы поднялась температура,0
2,кетонал инструкция по применению,тошнота как избавиться,0
3,лобода песня суперзвезда клип,как скачать песню на айфон,0
4,цинктерал стоимость в аптеке,если мало есть и заниматься спортом можно ли п...,0
...,...,...,...
495,почему армянки красивые,отучить котенка писать на кровать,0
496,борщевик сосновского история создания,как хранить деньги в кошельке,0
497,солнечный берег чувашия новый год,жирная кожа лица что делать,0
498,вязаная шапка из толстых ниток,как ломается голос у девочек,0


In [ ]:
ans_df.to_csv(path+'test_dataset_labeled.csv')
ddd = pd.read_csv(path+'test_dataset_labeled.csv')
ddd

,Unnamed: 0,query1,query2,label
0,0,как царапать глаза,как сделать двойную веку,0
1,1,основные методы реабилитации пороках сердца,что надо сделать чтобы поднялась температура,0
2,2,кетонал инструкция по применению,тошнота как избавиться,0
3,3,лобода песня суперзвезда клип,как скачать песню на айфон,0
4,4,цинктерал стоимость в аптеке,если мало есть и заниматься спортом можно ли п...,0
...,...,...,...,...
495,495,почему армянки красивые,отучить котенка писать на кровать,0
496,496,борщевик сосновского история создания,как хранить деньги в кошельке,0
497,497,солнечный берег чувашия новый год,жирная кожа лица что делать,0
498,498,вязаная шапка из толстых ниток,как ломается голос у девочек,0
